## Import Libraries

In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
import scipy.io as sio

from brainpipe.system import study
from brainpipe.statistics import *
from utils import rename_elecs

## Power Decoding - Partial//Detailed Encoding
### For ALL time points

In [2]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

st = study('Olfacto')
exp, meth = 'Enc', 'BETWEEN'#'Enc'
conds, subjects = ['low','high'],['PIRJ','FERJ','CHAF','LEFC','VACJ','SEMC']
freqs = ['high_gamma']
path_pow = path.join(st.path, 'feature/TPSim_2groups_Enc/TPS_btw_v=1_elecs=all/')
#filename = path.join(path_pow, 'TPS_spear_{}_cond_{}_{}_newscore_6freqs_3s_dissim.npz')
filename = path.join(path_pow, 'TPS_pears_{}_{}_btw_{}.npz')
save_path = path.join(st.path, 'classified/TPSim_clf_all_HG/')
#su, method(global,btw,wth),exp (E,R,ER),freq
if exp == 'Enc_Ret':
    name_classif = path.join(save_path, 
            '{}_LDA_clf_'+meth+'_E_R_newscore_'+conds[0]+'_'+conds[1]+'_{}_k=5.npz')
else:
    name_classif = path.join(save_path, 
         '{}_LDA_clf_'+meth+'_'+exp[0]+'_newscore_'+conds[0]+'_'+conds[1]+'_{}_k=5.npz')
rois_to_keep = ['ACC','aHC','IFG','Ins_olf','MFG','OFC_olf','PHG','SFG','pPirT','Amg']

nperm = 1000

if not path.exists(save_path):
    makedirs(save_path)

for su,freq in product(subjects,freqs):
    #for freq in freqs:
    pow_list = []
    #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
    mat0 = np.load(filename.format(su,conds[0],freq),allow_pickle=True)
    labels = mat0['label']
    x,y,z = mat0['xyz'][:,0], mat0['xyz'][:,1], mat0['xyz'][:,2]
    new_labels = rename_elecs(labels,x,y,z)
    idx_rois = np.isin(new_labels, rois_to_keep)
    
    pow_list.append(mat0['tps'][idx_rois])
    nelecs = mat0['tps'][idx_rois].shape[0]
    mat1 = np.load(filename.format(su,conds[1],freq),allow_pickle=True)
    print(mat1.files)
    pow_list.append(mat1['tps'][idx_rois])
    print (su,mat0.files, 'TPS shape: ', [pow.shape for pow in pow_list])

    #=========================== Create dict for all results =================================    
    #if not path.exists(name_classif.format(su,freq)):
    kwargs = {}
    kwargs['names'], kwargs['channels'] = new_labels[idx_rois], mat0['channel'][idx_rois]
    kwargs['xyz'] = mat0['xyz'][idx_rois]

    # =========================== Select Power for 1 elec 1 freq =================================                 
    permut, auc = np.zeros((nelecs,nperm)),np.zeros((nelecs))
    for elec_num in range(nelecs):
        print('--» processing',su, 'elec', elec_num,'/',nelecs)
        pow_data_elec = [power[elec_num][:,np.newaxis] for power in pow_list]

        # create a data matrix, concatenate along the trial dimension
        x = np.concatenate(pow_data_elec, axis=0)
        print ('Size of the concatenated data: ', x.shape)
        y = np.hstack([np.array([i]*len(power)) for i, power in enumerate(pow_data_elec)])
        print ('Size of label for classif: ', len(y))

        clf = LDA()
        cv = SKFold(5)
        score, permutation_scores, pvalue = permutation_test_score(
        clf, x, y, scoring="roc_auc", cv=cv, n_permutations=nperm, n_jobs=-1)
        permut[elec_num] += permutation_scores
        auc[elec_num] += score
        print("Classification score %s (pvalue : %s)" % (score, pvalue))

    kwargs['perm'], kwargs['auc'] = permut, auc
    #Save plots
    np.savez(name_classif.format(su,freq), **kwargs)
    del x, y, pow_data_elec, permutation_scores, score
    #else:
    #    print(name_classif.format(su,freq), 'already computed')
    del pow_list

-> Olfacto loaded
['tps', 'pval', 'label', 'channel', 'xyz']
PIRJ ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  [(24, 91), (24, 55)]
--» processing PIRJ elec 0 / 24
Size of the concatenated data:  (146, 1)
Size of label for classif:  146
Classification score 0.5313131313131313 (pvalue : 0.45554445554445555)
--» processing PIRJ elec 1 / 24
Size of the concatenated data:  (146, 1)
Size of label for classif:  146
Classification score 0.3930356193514089 (pvalue : 0.935064935064935)
--» processing PIRJ elec 2 / 24
Size of the concatenated data:  (146, 1)
Size of label for classif:  146
Classification score 0.39595959595959596 (pvalue : 0.9400599400599401)
--» processing PIRJ elec 3 / 24
Size of the concatenated data:  (146, 1)
Size of label for classif:  146
Classification score 0.567304625199362 (pvalue : 0.24275724275724275)
--» processing PIRJ elec 4 / 24
Size of the concatenated data:  (146, 1)
Size of label for classif:  146
Classification score 0.4308346624136098 (pvalue : 0.

Classification score 0.6293618986055961 (pvalue : 0.002997002997002997)
--» processing FERJ elec 22 / 32
Size of the concatenated data:  (237, 1)
Size of label for classif:  237
Classification score 0.4297940714747437 (pvalue : 0.7812187812187812)
--» processing FERJ elec 23 / 32
Size of the concatenated data:  (237, 1)
Size of label for classif:  237
Classification score 0.5283830455259027 (pvalue : 0.42357642357642356)
--» processing FERJ elec 24 / 32
Size of the concatenated data:  (237, 1)
Size of label for classif:  237
Classification score 0.6045304275556378 (pvalue : 0.014985014985014986)
--» processing FERJ elec 25 / 32
Size of the concatenated data:  (237, 1)
Size of label for classif:  237
Classification score 0.7180718441222643 (pvalue : 0.000999000999000999)
--» processing FERJ elec 26 / 32
Size of the concatenated data:  (237, 1)
Size of label for classif:  237
Classification score 0.7576950780312124 (pvalue : 0.000999000999000999)
--» processing FERJ elec 27 / 32
Size of 

Classification score 0.6126984126984127 (pvalue : 0.14985014985014986)
--» processing CHAF elec 36 / 61
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.3714285714285715 (pvalue : 0.8131868131868132)
--» processing CHAF elec 37 / 61
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.4000000000000001 (pvalue : 0.7352647352647352)
--» processing CHAF elec 38 / 61
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.7142857142857144 (pvalue : 0.013986013986013986)
--» processing CHAF elec 39 / 61
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.4412698412698413 (pvalue : 0.6143856143856143)
--» processing CHAF elec 40 / 61
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.7015873015873015 (pvalue : 0.014985014985014986)
--» processing CHAF elec 41 / 61
Size of the 

Classification score 0.6143959574980435 (pvalue : 0.000999000999000999)
--» processing LEFC elec 21 / 27
Size of the concatenated data:  (771, 1)
Size of label for classif:  771
Classification score 0.4778343417012823 (pvalue : 0.6703296703296703)
--» processing LEFC elec 22 / 27
Size of the concatenated data:  (771, 1)
Size of label for classif:  771
Classification score 0.46884030160736867 (pvalue : 0.7882117882117882)
--» processing LEFC elec 23 / 27
Size of the concatenated data:  (771, 1)
Size of label for classif:  771
Classification score 0.5722935855758233 (pvalue : 0.004995004995004995)
--» processing LEFC elec 24 / 27
Size of the concatenated data:  (771, 1)
Size of label for classif:  771
Classification score 0.5182905905725123 (pvalue : 0.3756243756243756)
--» processing LEFC elec 25 / 27
Size of the concatenated data:  (771, 1)
Size of label for classif:  771
Classification score 0.4845403648184938 (pvalue : 0.5684315684315684)
--» processing LEFC elec 26 / 27
Size of the 

Classification score 0.6374039938556068 (pvalue : 0.005994005994005994)
--» processing SEMC elec 1 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.7219278033794162 (pvalue : 0.000999000999000999)
--» processing SEMC elec 2 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.6873655913978494 (pvalue : 0.000999000999000999)
--» processing SEMC elec 3 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.4430875576036867 (pvalue : 0.7242757242757243)
--» processing SEMC elec 4 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.5532258064516128 (pvalue : 0.29270729270729273)
--» processing SEMC elec 5 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.5716205837173579 (pvalue : 0.16783216783216784)
--» processing SEMC elec 6 / 53
Size of the con

Classification score 0.6028417818740399 (pvalue : 0.05094905094905095)
--» processing SEMC elec 48 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.5146313364055299 (pvalue : 0.5134865134865135)
--» processing SEMC elec 49 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.5969662058371735 (pvalue : 0.058941058941058944)
--» processing SEMC elec 50 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.5286482334869431 (pvalue : 0.4825174825174825)
--» processing SEMC elec 51 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.3928955453149002 (pvalue : 0.936063936063936)
--» processing SEMC elec 52 / 53
Size of the concatenated data:  (501, 1)
Size of label for classif:  501
Classification score 0.4049923195084485 (pvalue : 0.8331668331668332)
